In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]") #,'lib/spark_PCA.py'

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [5]:
special_events = pd.read_csv('Datasources/other general/special_events_list_datasd.csv')
active_businesses = pd.read_csv('Datasources/other general/sd_active_businesses_datasd.csv')

In [6]:
special_events

event_title  event_id  \
0              2018 Hillcrest Farmers' Market (Sundays)     49953   
1                             Old Town Artisan's Market     50424   
2                                 Sunday Artisan Market     50588   
3                             Old Town Artisan's Market     50423   
4         City Heights Farmer's Market (Every Saturday)     50130   
5            The Mercato Little Italy's Farmers' Market     50028   
6                                       Holiday Bowl 5K     49893   
7                            North Park Thursday Market     50078   
8     Ocean Beach Certified Farmer's Market (Wednesd...     50212   
9                             Old Town Artisan's Market     50422   
10                                Sunday Artisan Market     50587   
11             2018 Hillcrest Farmers' Market (Sundays)     49952   
12                            Old Town Artisan's Market     50421   
13        City Heights Farmer's Market (Every Saturday)     50129   
14           The Mercato Little Italy's Farmers' Market     50027   
15                           North Park Thursday Market     50077   
16    Ocean Beach Certified Farmer's Market (Wednesd...     50211   
17    Curbside Bites Food Truck Markets - Downtown L...     49831   
18                Pacific Beach Tuesday Farmers' Market     50505   
19                                Sunday Artisan Market     50586   
20                            Old Town Artisan's Market     50420   
21             2018 Hillcrest Farmers' Market (Sundays)     49951   
22                                   Hillcrest Car Show     50154   
23        City Heights Farmer's Market (Every Saturday)     50128   
24                            Old Town Artisan's Market     50419   
25           The Mercato Little Italy's Farmers' Market     50026   
26                           North Park Thursday Market     50076   
27    Ocean Beach Certified Farmer's Market (Wednesd...     50210   
28    Curbside Bites Food Truck Markets - Downtown L...     49830   
29                Pacific Beach Tuesday Farmers' Market     50504   
...                                                 ...       ...   
2737                          Old Town Artisan's Market     47987   
2738                                     EarthFair 2016     48105   
2739   Third Avenue Certified Farmer's Market (Sundays)     47495   
2740           2016 Hillcrest Farmers' Market (Sundays_     47382   
2741                             Jacaranda Spring Thing     48152   
2742                          Old Town Artisan's Market     47986   
2743              Golden Hill Certified Farmer's Market     47798   
2744      City Heights Farmer's Market (Every Saturday)     47441   
2745                  Fighting Parkinson's Step by Step     48100   
2746  The Mercato: Little Italy's Farmers Market (Sat.)     47330   
2747                            The BLVD Monthly Market     47703   
2748                         North Park Farmer's Market     47904   
2749               UTC Farmer's Market (Every Thursday)     47850   
2750  Ocean Beach Certified Farmer's Market (Wednesd...     47650   
2751     Curbside Bites Food Truck Markets (Wed. Lunch)     47598   
2752              Pacific Beach Tuesday Farmer's Market     47546   
2753                         RIMS Corporate Blcok Party     48094   
2754                          Old Town Artisan's Market     47985   
2755   Third Avenue Certified Farmer's Market (Sundays)     47494   
2756           2016 Hillcrest Farmers' Market (Sundays_     47381   
2757           12th Annual La Jolla Concours d'Elegance     48099   
2758                    Fiesta Island Time Trial Series     47718   
2759                         Rum Run 5K & Kids Tiki Run     48171   
2760         8th Annual GranFondo San Diego (Bike Ride)     47761   
2761             Ray at Night (2nd Saturday Each Month)     47891   
2762                          Old Town Artisan's Market     47984   
2763              Golden Hill Certified Fa

In [7]:
active_businesses

account_key                address_str apt_suite bus_start_dt  \
0       2017014278   10640 scripps ranch blvd       101   2017-08-23   
1       2017013866          5085 september st       NaN   2015-09-01   
2       2017013873               827 beryl st       NaN   2015-09-01   
3       2017013871            2704 boston ave       NaN   2015-09-01   
4       2017013870         1502 mesa brook st       NaN   2015-09-01   
5       2017013869        4775 monongahela st       NaN   2015-09-01   
6       2017013867          302 washington st    150-31   2015-09-01   
7       2017013865           2464 fenton pkwy       207   2015-09-01   
8       2017013874          2616 highland ave         5   2015-09-01   
9       2017013863                  4584 f st       NaN   2015-09-01   
10      2017013862           3456 marathon dr       NaN   2015-09-01   
11      2017013861             4840 baylor dr       NaN   2015-09-01   
12      2017013860            5353 topsail dr       NaN   2015-09-01   
13      2017013859      6637 green gables ave       NaN   2015-09-01   
14      2017013872            9520 easter way       NaN   2015-09-01   
15      2017013875         11570 windcrest ln      1612   2015-09-01   
16      2017013856             3828 centre st       NaN   2015-09-01   
17      2017013884           6403 seascape dr       NaN   2015-09-01   
18      2017013889     2511 caminito avellano       NaN   2015-09-01   
19      2017013890            5714 roswell st       NaN   2015-09-01   
20      2017013888            4422 keeler ave       NaN   2015-09-01   
21      2017013887         3003 fairmount ave         3   2015-09-01   
22      2017013886                  3770 t st       NaN   2015-09-01   
23      2017013885                711 61st st         p   2015-09-01   
24      2017013883          5949 alleghany st       NaN   2015-09-01   
25      2017013882            1154 bittern st       NaN   2015-09-01   
26      2017013880              4095 texas st         1   2015-09-01   
27      2017013881             3144 haller st       NaN   2015-09-01   
28      2017013879             3595 agosto st       NaN   2015-09-01   
29      2017013878                230 jojo ct       NaN   2015-09-01   
...            ...                        ...       ...          ...   
75423   1974010673          3145 rosecrans st         e   1974-07-01   
75424   1974004390           1413 morena blvd       NaN   1974-07-01   
75425   1974011543              9340 dowdy dr       102   1974-07-01   
75426   1974012203  8888 clairemont mesa blvd         m   1974-07-01   
75427   1974009442    1219 hidden mountain dr       NaN   1974-07-01   
75428   1974009118              5228 riley st       NaN   1974-07-01   
75429   1974010345             6810 curran st       NaN   1974-07-01   
75430   1974001703             3626 ruffin rd       NaN   1974-07-01   
75431   1974005190            7812 girard ave       NaN   1974-07-01   
75432   1974010051          5534 red river dr       NaN   1974-07-01   
75433   1974005140               520 pearl st       NaN   1974-07-01   
75434   1974010644                   740 f st       NaN   1974-07-01   
75435   1974010929           2521 pacific hwy       NaN   1974-07-01   
75436   1974012790           2335 morena blvd       NaN   1974-07-01   
75437   1974012733             286 euclid ave       NaN   1974-07-01   
75438   1974009274             4787 cardin st       NaN   1974-07-01   
75439   1974015236      223 via de san ysidro         4   1974-07-01   
75440   1974011425             4050 adams ave       NaN   1974-07-01   
75441   1974009340        7255 university ave       NaN   1974-07-01   
75442   1974011168          2104 national ave       NaN   1974-07-01   
75443   1974005390     3220 sports arena blvd       NaN   1974-07-01   
75444   1974016111        1342 otay valley rd       NaN   1974-07-01   
75445   1974009389                  1130 d st         7   1974-07-01   
75446   1974016058    6429 caminito baltusral  

In [8]:
listings

id                            listing_url       scrape_id  \
0     11204286  https://www.airbnb.com/rooms/11204286  20160706203047   
1      7972006   https://www.airbnb.com/rooms/7972006  20160706203047   
2      7727710   https://www.airbnb.com/rooms/7727710  20160706203047   
3     13124681  https://www.airbnb.com/rooms/13124681  20160706203047   
4      3469225   https://www.airbnb.com/rooms/3469225  20160706203047   
5       877473    https://www.airbnb.com/rooms/877473  20160706203047   
6      3124507   https://www.airbnb.com/rooms/3124507  20160706203047   
7      3432507   https://www.airbnb.com/rooms/3432507  20160706203047   
8      3249729   https://www.airbnb.com/rooms/3249729  20160706203047   
9     11756336  https://www.airbnb.com/rooms/11756336  20160706203047   
10    12173670  https://www.airbnb.com/rooms/12173670  20160706203047   
11    13157364  https://www.airbnb.com/rooms/13157364  20160706203047   
12     9756570   https://www.airbnb.com/rooms/9756570  20160706203047   
13    10687910  https://www.airbnb.com/rooms/10687910  20160706203047   
14     7938260   https://www.airbnb.com/rooms/7938260  20160706203047   
15    10779932  https://www.airbnb.com/rooms/10779932  20160706203047   
16    13301630  https://www.airbnb.com/rooms/13301630  20160706203047   
17     5285136   https://www.airbnb.com/rooms/5285136  20160706203047   
18    13313236  https://www.airbnb.com/rooms/13313236  20160706203047   
19     7663518   https://www.airbnb.com/rooms/7663518  20160706203047   
20     5077385   https://www.airbnb.com/rooms/5077385  20160706203047   
21     7113838   https://www.airbnb.com/rooms/7113838  20160706203047   
22     8555262   https://www.airbnb.com/rooms/8555262  20160706203047   
23    13472856  https://www.airbnb.com/rooms/13472856  20160706203047   
24     4997701   https://www.airbnb.com/rooms/4997701  20160706203047   
25     6098460   https://www.airbnb.com/rooms/6098460  20160706203047   
26    13562988  https://www.airbnb.com/rooms/13562988  20160706203047   
27    10284248  https://www.airbnb.com/rooms/10284248  20160706203047   
28    13693167  https://www.airbnb.com/rooms/13693167  20160706203047   
29    12988363  https://www.airbnb.com/rooms/12988363  20160706203047   
...        ...                                    ...             ...   
6578  13765978  https://www.airbnb.com/rooms/13765978  20160706203047   
6579  13637335  https://www.airbnb.com/rooms/13637335  20160706203047   
6580   7462953   https://www.airbnb.com/rooms/7462953  20160706203047   
6581  13816897  https://www.airbnb.com/rooms/13816897  20160706203047   
6582    181308    https://www.airbnb.com/rooms/181308  20160706203047   
6583  12485414  https://www.airbnb.com/rooms/12485414  20160706203047   
6584  13710734  https://www.airbnb.com/rooms/13710734  20160706203047   
6585  13868104  https://www.airbnb.com/rooms/13868104  20160706203047   
6586  13526039  https://www.airbnb.com/rooms/13526039  20160706203047   
6587  13689250  https://www.airbnb.com/rooms/13689250  20160706203047   
6588  13495353  https://www.airbnb.com/rooms/13495353  20160706203047   
6589  10328997  https://www.airbnb.com/rooms/10328997  20160706203047   
6590   9710574   https://www.airbnb.com/rooms/9710574  20160706203047   
6591  13448725  https://www.airbnb.com/rooms/13448725  20160706203047   
6592  13864897  https://www.airbnb.com/rooms/13864897  20160706203047   
6593  13859950  https://www.airbnb.com/rooms/13859950  20160706203047   
6594  11375621  https://www.airbnb.com/rooms/11375621  20160706203047   
6595  13443354  https://www.airbnb.com/rooms/13443354  20160706203047   
6596  13670544  https://www.airbnb.com/rooms/13670544  20160706203047   
6597   8271640   https://www.airbnb.com/rooms/8271640  20160706203047   
6598  13411765  https://www.airbnb.com/rooms/13411765  20160706203047   
6599  13377361  https://www.airbnb.com/rooms/13377361  20160706203047   
6600  13466129  https://www.airbnb.com/rooms/13466129  20160706203047 

In [9]:
listings[['latitude', 'longitude']]

latitude   longitude
0     32.581882 -117.001940
1     32.653029 -116.976812
2     32.661477 -116.971813
3     32.638694 -116.951252
4     32.638942 -116.938891
5     32.638285 -116.939373
6     32.637183 -116.934752
7     32.637594 -116.935737
8     32.637588 -116.934959
9     32.635328 -116.958756
10    32.647356 -116.955336
11    32.634342 -116.953567
12    32.628250 -116.961313
13    32.627788 -116.959399
14    32.627834 -117.027535
15    32.628429 -117.027548
16    32.628160 -117.025583
17    32.633652 -117.031437
18    32.628557 -117.027218
19    32.626812 -116.997218
20    32.617235 -116.955314
21    32.624131 -116.959827
22    32.616518 -117.009670
23    32.626928 -116.989164
24    32.617546 -116.956672
25    32.637008 -116.985868
26    32.615972 -116.971834
27    32.625552 -117.000635
28    32.620946 -116.978838
29    32.626589 -116.989851
...         ...         ...
6578  32.949464 -117.263159
6579  32.755944 -117.047831
6580  32.688773 -117.071752
6581  32.651841 -117.109818
6582  32.662955 -117.049579
6583  32.628783 -116.963484
6584  32.788481 -117.013504
6585  32.762549 -117.196074
6586  32.765321 -117.249739
6587  32.650229 -117.108413
6588  32.650971 -117.007079
6589  32.532561 -117.116331
6590  32.720057 -117.221661
6591  32.802779 -117.260800
6592  32.950118 -117.265119
6593  32.624830 -117.101989
6594  32.979508 -117.258232
6595  33.083107 -117.040519
6596  32.956821 -117.260117
6597  33.005791 -117.104335
6598  32.679279 -117.027502
6599  32.733057 -117.057120
6600  32.835511 -117.001812
6601  32.946526 -117.260963
6602  32.532181 -117.116172
6603  32.949025 -117.262833
6604  32.985937 -117.253628
6605  32.979522 -117.257123
6606  32.985098 -117.259575
6607  32.980441 -117.258719

[6608 rows x 2 columns]

In [11]:
special_events_df = sqlContext.createDataFrame(open('Datasources/other general/special_events_list_datasd.csv','rb'))
active_businesses_df = sqlContext.createDataFrame(open('Datasources/other general/sd_active_businesses_datasd.csv','rb'))

TypeError: Can not infer schema for type: <type 'str'>

In [ ]:
df = pd.DataFrame()
count = 0

for idx_one in listings.index:
    for idx_two in special_events.index:
        p_one = float(listings['latitude'][idx_one])
        q_one = float(listings['longitude'][idx_one])
        p_two = float(special_events['lat'][idx_two])
        q_two = float(special_events['lon'][idx_two])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371*float(c)
        
        k = pd.DataFrame.from_dict({count: {'listing_id': listings['id'][idx_one], 
                                            'event_title': special_events['event_title'][idx_two],
                                             'event_id': special_events['event_id'][idx_two],
                                           'distance_from_event': d}})
        df = df.append(k)
        
        count = count + 1

In [ ]:
df

In [ ]:
df_two = pd.DataFrame()
count = 0

for idx_one in listings.index:
    for idx_two in active_businesses.index:
        p_one = float(listings['latitude'][idx_one])
        q_one = float(listings['longitude'][idx_one])
        p_two = float(active_businesses['latitude'][idx_two])
        q_two = float(active_businesses['longitude'][idx_two])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371*float(c)
        
        k = pd.DataFrame.from_dict({count: {'listing_id': listings['id'][idx_one], 
                                            'name_of_company': active_businesses['doing_bus_as_name'][idx_two],
                                             'account_key': active_businesses['account_key'][idx_two],
                                           'distance_from_business': d}})
        
        df_two = df_two.append(k)
        
        count = count + 1

In [ ]:
df_two